# 1. 自动识别文件

In [1]:
import os
import datetime

In [2]:
list_file = list(os.walk('./'))[0][2]
for file in list_file:
    if 'Lennon_整理好的替代料总表' in file:
        alternate = file
    elif '改价表' in file:
        stock = file
    elif 'US MRP' in file:
        mrp = file

print(f'alternate: \t{alternate}')
print(f'stock: \t\t{stock}')
print(f'mrp: \t\t{mrp}')

alternate: 	Lennon_整理好的替代料总表_20230822_140006.xlsx
stock: 		改价表 8-22.xlsx
mrp: 		US MRP20230821-运营.xlsx


# 2. 计算库存
## 2.1 替代料库存

In [3]:
import pandas as pd

In [4]:
df_alternate = pd.read_excel(alternate,
                             header=0).fillna('')
df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6
0,USCA171876D-3,USCA171876D,USCA171876,USCA171876-3,,,
1,USGS37022G,USGS37022T,USGS37022,USGS37022J,,,
2,USGS36761T,USGS36761TB,USGS36761,,,,
3,USGS41570B,USGS41570,,,,,
4,USGS69025S,USGS69025,USGS69025T,USGS69025TB,,,
...,...,...,...,...,...,...,...
40946,USTLA85200-C,USTLA85200,,,,,
40947,USTLA89011,USTLA89011-C,,,,,
40948,USWR40588,USWR40588-C,,,,,
40949,USWR40589-C,USWR40589,,,,,


In [5]:
df_alternate['row'] = [str(i) for i in range(len(df_alternate))]

df_alternate

,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,row
0,USCA171876D-3,USCA171876D,USCA171876,USCA171876-3,,,,0
1,USGS37022G,USGS37022T,USGS37022,USGS37022J,,,,1
2,USGS36761T,USGS36761TB,USGS36761,,,,,2
3,USGS41570B,USGS41570,,,,,,3
4,USGS69025S,USGS69025,USGS69025T,USGS69025TB,,,,4
...,...,...,...,...,...,...,...,...
40946,USTLA85200-C,USTLA85200,,,,,,40946
40947,USTLA89011,USTLA89011-C,,,,,,40947
40948,USWR40588,USWR40588-C,,,,,,40948
40949,USWR40589-C,USWR40589,,,,,,40949


In [6]:
df_stock = pd.read_excel(stock,
                         header=0,
                         names=['SKU', '当日库存'],
                         usecols='A, F')

df_stock

,SKU,当日库存
0,USBDA04239,0
1,USBDA04868,0
2,USDLA3ZP50,0
3,USEVC271FG,0
4,USBDA04007,1
...,...,...
241935,USGS49820,1
241936,USWR52564C,1
241937,USCVA63523-C,2
241938,USRAD13576-C,4


In [7]:
from tqdm import tqdm

In [8]:
for i in tqdm(range(len(df_stock)), desc='运行中', ncols=77):
    if df_stock.loc[i, 'SKU'].endswith('-NC'):
        df_stock = df_stock.drop(i)

运行中: 100%|█████████████████████| 241940/241940 [00:10<00:00, 22646.61it/s]


In [9]:
df_stock = df_stock.reset_index(drop=True)
df_stock['SKU'] = [df_stock.loc[i, 'SKU'][:-2] if df_stock.loc[i, 'SKU'].endswith('-N') else df_stock.loc[i, 'SKU'] for i in range(len(df_stock))]
df_stock = df_stock.groupby([df_stock['SKU']], as_index=False).sum()

df_stock

,SKU,当日库存
0,CAWR41163,1
1,CS20230609,0
2,CS230530,0
3,CSPJ20230615,1
4,CSPJ230609,0
...,...,...
73014,YHTWS58202-Y-6,0
73015,YHTWS5W1ZBA-Y-10,0
73016,YHTWS5W1ZBA-Y-8,0
73017,YHTWS802AA-Y-6,0


In [10]:
df_alternate['当日库存'] = 0
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='主料SKU', right_on='SKU', suffixes=('', '_0')).fillna(0)
print('stock - 主料SKU')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料1', right_on='SKU', suffixes=('', '_1')).fillna(0)
print('stock - 替换料1')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料2', right_on='SKU', suffixes=('', '_2')).fillna(0)
print('stock - 替换料2')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料3', right_on='SKU', suffixes=('', '_3')).fillna(0)
print('stock - 替换料3')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料4', right_on='SKU', suffixes=('', '_4')).fillna(0)
print('stock - 替换料4')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料5', right_on='SKU', suffixes=('', '_5')).fillna(0)
print('stock - 替换料5')
df_alternate = pd.merge(left=df_alternate, right=df_stock, how='left', left_on='替换料6', right_on='SKU', suffixes=('', '_6')).fillna(0)
print('stock - 替换料6')

df_alternate['当日库存'] = df_alternate['当日库存_0'] + df_alternate['当日库存_1'] + df_alternate['当日库存_2'] + df_alternate['当日库存_3'] + df_alternate['当日库存_4'] + df_alternate['当日库存_5'] + df_alternate['当日库存_6']
df_alternate = df_alternate[['主料SKU',
                             '替换料1',
                             '替换料2',
                             '替换料3',
                             '替换料4',
                             '替换料5',
                             '替换料6',
                             '当日库存']]

df_alternate

stock - 主料SKU
stock - 替换料1
stock - 替换料2
stock - 替换料3
stock - 替换料4
stock - 替换料5
stock - 替换料6


,主料SKU,替换料1,替换料2,替换料3,替换料4,替换料5,替换料6,当日库存
0,USCA171876D-3,USCA171876D,USCA171876,USCA171876-3,,,,12.0
1,USGS37022G,USGS37022T,USGS37022,USGS37022J,,,,18.0
2,USGS36761T,USGS36761TB,USGS36761,,,,,4.0
3,USGS41570B,USGS41570,,,,,,110.0
4,USGS69025S,USGS69025,USGS69025T,USGS69025TB,,,,33.0
...,...,...,...,...,...,...,...,...
40946,USTLA85200-C,USTLA85200,,,,,,0.0
40947,USTLA89011,USTLA89011-C,,,,,,0.0
40948,USWR40588,USWR40588-C,,,,,,0.0
40949,USWR40589-C,USWR40589,,,,,,0.0


In [11]:
list_df = []
for i in ['主料SKU', '替换料1', '替换料2', '替换料3', '替换料4', '替换料5', '替换料6']:
    df_temp =  df_alternate[[i,
                             '当日库存']]
    df_temp.columns = ['SKU',
                       '当日库存']
    list_df.append(df_temp)

df_position = pd.concat(list_df)
df_position = df_position[df_position['SKU'] != ''].reset_index(drop=True)

df_position

,SKU,当日库存
0,USCA171876D-3,12.0
1,USGS37022G,18.0
2,USGS36761T,4.0
3,USGS41570B,110.0
4,USGS69025S,33.0
...,...,...
86788,USGS27875,1.0
86789,USGS16178GNZB-C,0.0
86790,USGS6310TNZB-C,27.0
86791,USIC2A367A,5.0


## 2.2 MRP表库存

In [12]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       usecols='B').fillna(0)

df_mrp

,MRP SKU
0,USTCV17215HIE-2-C
1,USTBC40007-C
2,USBDA04108-2-C
3,USET87039H-C
4,USGSM3570T-C
...,...
56520,USWSP65508B1-2-YC
56521,USWSP54521B1-4-YC
56522,USWSP54717B1-4-YC
56523,USWSP54717B1-2-YC


In [13]:
df_mrp = pd.merge(left=df_mrp, right=df_stock, how='left', left_on='MRP SKU', right_on='SKU').fillna(0)
df_mrp = pd.merge(left=df_mrp, right=df_position, how='left', left_on='MRP SKU', right_on='SKU', suffixes=('_stock', '_position')).fillna('')

df_mrp['库存'] = [df_mrp.loc[i, '当日库存_position'] if df_mrp.loc[i, '当日库存_position'] != '' else df_mrp.loc[i, '当日库存_stock'] for i in range(len(df_mrp))]
df_mrp = df_mrp[['MRP SKU',
                 '库存']]

df_mrp

,MRP SKU,库存
0,USTCV17215HIE-2-C,123.0
1,USTBC40007-C,248.0
2,USBDA04108-2-C,126.0
3,USET87039H-C,118.0
4,USGSM3570T-C,240.0
...,...,...
56520,USWSP65508B1-2-YC,0.0
56521,USWSP54521B1-4-YC,0.0
56522,USWSP54717B1-4-YC,0.0
56523,USWSP54717B1-2-YC,0.0


# 3. 保存

In [14]:
import datetime

In [15]:
writer = pd.ExcelWriter('Lennon_整理好的库存表_' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx')
df_mrp.to_excel(writer, index=False)
writer.close()